In [1]:
!pip install open_clip_torch pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 kB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 93.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 93.3 MB/s eta 0:00:00


In [2]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score,f1_score
from sklearn.preprocessing import LabelEncoder 
import open_clip
pl.seed_everything(56)

Seed set to 56


56

In [3]:
!pip install transformers -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 81.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 108.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 58.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.1.0
    Uninstalling fsspec-2023.1.0:
      Successfully uninstalled fsspec-2023.1.0
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.12.0
    Uninstalling huggingface-hub-0.12.0:
      Successfully uninstalled huggingface-hub-0.12.0
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Attempting uninstall: transformers
    Found existing ins

In [3]:
!nvidia-smi

Sat Nov 11 19:27:31 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:00:05.0 Off |                  Off |
| 30%   32C    P8    20W / 300W |      1MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
open_clip.list_pretrained()

[('RN50', 'openai'),
 ('RN50', 'yfcc15m'),
 ('RN50', 'cc12m'),
 ('RN50-quickgelu', 'openai'),
 ('RN50-quickgelu', 'yfcc15m'),
 ('RN50-quickgelu', 'cc12m'),
 ('RN101', 'openai'),
 ('RN101', 'yfcc15m'),
 ('RN101-quickgelu', 'openai'),
 ('RN101-quickgelu', 'yfcc15m'),
 ('RN50x4', 'openai'),
 ('RN50x16', 'openai'),
 ('RN50x64', 'openai'),
 ('ViT-B-32', 'openai'),
 ('ViT-B-32', 'laion400m_e31'),
 ('ViT-B-32', 'laion400m_e32'),
 ('ViT-B-32', 'laion2b_e16'),
 ('ViT-B-32', 'laion2b_s34b_b79k'),
 ('ViT-B-32', 'datacomp_xl_s13b_b90k'),
 ('ViT-B-32', 'datacomp_m_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_clip_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_laion_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_image_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_text_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_basic_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_s128m_b4k'),
 ('ViT-B-32', 'datacomp_s_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_clip_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_laion_s13m_b4k'),
 ('ViT-B-32', 'commonpool_

In [6]:
class CFG:
    wandb=False
    num_workers=8
    model="ViT-B-16-SigLIP"
    pretrained = "webli"
    #model = 'ViT-B-32'
    #pretrained = 'commonpool_m_clip_s128m_b4k'
    hidden_size = 768 * 2
    train_path='translated_trainv2.csv'
    test_path = './train-test-csvs/test-5.csv'
    val_split_size = 0.2
    num_labels = 8
    scheduler='cosine'
    max_epoches=1
    clip_lr = 1e-6
    mlp_lr = 5e-5
    text_lr = 2e-5
    eps=1e-6
    weights_decay=0.01
    batch_size=4
    seed=56

In [7]:
class PLDataset(torch.utils.data.Dataset):
    def __init__(self, df,preprocess,tokenizer):
        super().__init__()
        self.cfg = CFG()
        self.data = df[['image','label','text']]
        self.data = self.data.values
        self.processor = preprocess
        self.tokenizer = tokenizer
    def __getitem__(self, index):
        image = Image.open(self.data[index][0])
        image = self.processor(image)
        text = self.tokenizer(self.data[index][2])
        label = self.data[index][1]
        return image,text[0],label
    def __len__(self):
        return len(self.data)

In [8]:
le = LabelEncoder()
label_vc = {'Развлечения и юмор':0,
            'Кулинария':1,
            'Торговля и объявления':2,
            'СМИ':3,
            'Философия и религия':4,
            'Животные':5,
            'Творчество и дизайн':6,
            'Путешествия':7}

def process_labels(label):
    if label not in label_vc.keys():
        return -1
    else:
        return label_vc[label]
def process_text(text:str):
    return text.strip().lower()

def make_df(path):
    df = pd.DataFrame()
    data = pd.read_csv(path,sep=';')
    if 'label' in data.columns:
        data['label'] = data['label'].map(process_labels)
        data = data[data['label'] != -1].reset_index()
        df['text'] = data[data['label'] != -1]['description'].fillna('')
        df['label'] = le.fit_transform(data['label'])
        df['image'] = data['id'].map(lambda x:'./vseros-final-data/Data/Train/'+str(x))
    else:
        df['label'] = 0
        df['text'] = data['description'].fillna('')
        df['image'] = data['id'].map(lambda x:'./vseros-final-data/Data/Test/'+str(x))
    return df

In [9]:
class PLDataModule(pl.LightningDataModule):
    def __init__(self,processor,tokenizer):
        super().__init__()
        self.processor = processor
        self.tokenizer = tokenizer
        self.cfg = CFG()
        self.train_dataset_path = self.cfg.train_path
        self.test_dataset_path = self.cfg.test_path
        self.val_split_size = self.cfg.val_split_size
        self.batch_size = self.cfg.batch_size
        self.num_workers = self.cfg.num_workers
        self.is_setup = False
    def prepare_data(self):
        #self.train_df = make_df(self.train_dataset_path)
        self.train_df = pd.read_csv(self.train_dataset_path,sep=';')
        self.test_df = make_df(self.test_dataset_path)
    def setup(self, stage: str):
        if self.is_setup:
            return None
        self.train_df, self.val_df = train_test_split(self.train_df, test_size=self.val_split_size,random_state=self.cfg.seed)
        self.train_dataset = PLDataset(self.train_df,self.processor,self.tokenizer)
        self.val_dataset = PLDataset(self.val_df,self.processor,self.tokenizer)
        self.predict_dataset = PLDataset(self.test_df,self.processor,self.tokenizer)
        self.is_setup = True
    
    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_dataset,
                                           batch_size=self.batch_size,
                                           num_workers=self.num_workers,
                                           shuffle=True)
    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_dataset,
                                           batch_size=self.batch_size,
                                           num_workers=self.num_workers)

    def predict_dataloader(self):
        return torch.utils.data.DataLoader(self.predict_dataset,
                                           batch_size=self.batch_size,
                                           num_workers=self.num_workers,
                                           shuffle = False)

In [30]:
class PLModule(pl.LightningModule):
    def __init__(self,clip):
        super().__init__()
        self.cfg = CFG()
        self.clip = clip
        self.mlp = nn.Sequential(nn.Linear(self.cfg.hidden_size,self.cfg.hidden_size * 2),
                                 nn.LayerNorm(self.cfg.hidden_size * 2),
                                 nn.GELU(),
                                 nn.Linear(self.cfg.hidden_size * 2,self.cfg.num_labels))
        self.criterion = nn.CrossEntropyLoss()
        self.val_targets = []
        self.val_preds = []
        self.get_features = False
        
    def get_clip_features(self,x1,x2):
        image_features = self.clip.encode_image(x1)
        text_features = self.clip.encode_text(x2)
        return torch.cat([image_features,text_features],axis=-1)
    def forward(self,x1,x2):
        features = self.get_clip_features(x1,x2)
        return self.mlp(features)

    def training_step(self, batch, _):
        x1,x2,targets = batch
        logits = self(x1,x2)
        loss = self.criterion(logits, targets)
        return loss
        
    def validation_step(self, batch, _):
        x1,x2,targets = batch
        logits = self(x1,x2).argmax(dim=-1).cpu().detach().tolist()
        self.val_targets += targets.tolist()
        self.val_preds += logits
        
    def predict_step(self, batch, _):
        x1,x2,_ = batch
        logits = self(x1,x2).softmax(dim=-1).cpu().detach().tolist()
        return logits
        
    def calc_metric(self):
        return 100 * (f1_score(self.val_targets,self.val_preds,average='macro'))
        
    def on_validation_epoch_end(self):
        print(self.calc_metric())
        self.val_targets, self.val_preds = [],[]
            
    def configure_optimizers(self):
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in self.clip.visual.named_parameters()],
                "lr": self.cfg.clip_lr
            },
            {
                "params": [p for n, p in self.clip.text.named_parameters()],
                "lr": self.cfg.clip_lr
            },
            {
                "params": [p for n, p in self.mlp.named_parameters()],
                "lr": self.cfg.mlp_lr
            }
        ]
        return torch.optim.AdamW(optimizer_grouped_parameters,
                                 weight_decay=self.cfg.weights_decay
                                )

In [11]:
clip, _, preprocess = open_clip.create_model_and_transforms(CFG.model, pretrained=CFG.pretrained)

In [12]:
tokenizer = open_clip.get_tokenizer(CFG.model)

In [13]:
dm = PLDataModule(preprocess,tokenizer)
dm.prepare_data()
dm.train_df['text'] = dm.train_df['text'].fillna('Without Description')
dm.setup(0)

In [31]:
model_pl = PLModule(clip)

In [32]:
lr_monitor = pl.callbacks.LearningRateMonitor(logging_interval='epoch')
checkpoint_cb = pl.callbacks.ModelCheckpoint(
    dirpath='./outputs/',
    filename='model_{epoch:02d}-{val_acc:.4f}',
    monitor='val_acc',
    mode='max',
    save_last=True
)

trainer = pl.Trainer(
    accelerator="gpu",
    precision=32,
    min_epochs=1,
    check_val_every_n_epoch=1,
    max_epochs=5
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [33]:
trainer.fit(model_pl,datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | clip      | CustomTextCLIP   | 203 M 
1 | mlp       | Sequential       | 4.8 M 
2 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
207 M     Trainable params
0         Non-trainable params
207 M     Total params
831.633   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

6.666666666666667


Training: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [34]:
test_preds = trainer.predict(model_pl,datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [36]:
test_preds = np.concatenate(test_preds)

In [43]:
test_preds.argmax(axis=-1)

array([0, 0, 7, ..., 7, 5, 7])

In [17]:
df = pd.DataFrame()

In [18]:
df['label'] = np.sum(test_preds)
df['id'] = pd.read_csv(CFG.test_path,sep=';').id

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [19]:
df['label'] = df['label'].apply(lambda x:list(label_vc.keys())[x])

In [20]:
df.to_csv('sfsfvt10.csv',index=False,sep=';')